In [14]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [15]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [16]:
def api_call_func(api_params: dict) -> list[dict]:

    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    fields_purchase_lines = ['product_id', 'price_unit', 'date_order', 'order_id']
    purchase_lines_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("date_order", ">=", (datetime.today() - timedelta(days = 100)).strftime("%Y-%m-%d"))]])
    purchase_lines_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_lines_ids], {'fields':fields_purchase_lines})


    fields_product_template = ['name', 'type', 'standard_price', 'list_price', 'write_date']
    product_template_ids = models.execute_kw(api_db, uid, api_clave, 'product.template', 'search', [[]])
    product_template_json = models.execute_kw(api_db, uid, api_clave, 'product.template', 'read', [product_template_ids], {'fields':fields_product_template})


    return purchase_lines_json, product_template_json

In [17]:
def purchase_lines_df_fun(purchase_lines_json: list[dict]) -> pd.DataFrame:

    data_purchase_lines = []

    for purchase in purchase_lines_json:
        new = {}
        new['purchase_line_id'] = purchase['id']
        new['product_id'] = purchase['product_id'][0]
        new['purchase_cost_unit'] = purchase['price_unit']
        new['purchase_date_order'] = purchase['date_order']
        new['purchase_order_id'] = purchase['order_id'][0]

        data_purchase_lines.append(new)

    purchase_lines_df1 = pd.DataFrame(data_purchase_lines)
    purchase_lines_df1['purchase_date_order'] = pd.to_datetime(purchase_lines_df1['purchase_date_order'], format='%Y-%m-%d %H:%M:%S')
    purchase_lines_df1['purchase_line_id'] = purchase_lines_df1['purchase_line_id'].astype('Int64')
    purchase_lines_df1['purchase_order_id'] = purchase_lines_df1['purchase_order_id'].astype('Int64')

    purchase_lines_df = purchase_lines_df1.loc[purchase_lines_df1['product_id'].drop_duplicates().index]

    return purchase_lines_df

In [18]:
def product_template_df_fun(product_template_json: list[dict]) -> pd.DataFrame:
    
    data_product_template = []

    for product in product_template_json:
        new = {}
        new['product_id'] = product['id']
        new['product_type'] = product['type']
        new['product_description'] = product['name']
        new['product_template_cost'] = product['standard_price']
        new['product_price'] = product['list_price']
        new['write_date'] = product['write_date']

        data_product_template.append(new)

    product_template_df = pd.DataFrame(data_product_template)
    product_template_df['write_date'] = pd.to_datetime(product_template_df['write_date'], format='%Y-%m-%d %H:%M:%S')
    product_template_df['product_id'] = product_template_df['product_id'].astype('Int64')

    return product_template_df

In [19]:
def complete_costo_df_fun(purchase_lines_df: pd.DataFrame, product_template_df: pd.DataFrame) -> pd.DataFrame:
    
    complete_costo_df = product_template_df.merge(purchase_lines_df, how='left', on='product_id')


    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_type'] = 'REPS'
    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_unit'] = complete_costo_df['product_template_cost']
    
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_type'] = 'Compra'
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_unit'] = complete_costo_df['purchase_cost_unit']
    
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_type'] = 'Servicio'
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_unit'] = pd.NA

    complete_costo_df['cost_unit'] = complete_costo_df['cost_unit'] * -1


    return complete_costo_df


In [20]:
def costo_func(test_db: bool = False) -> pd.DataFrame:
    
    api_params = api_params_func(test_db)
    purchase_lines_json, product_template_json = api_call_func(api_params)

    purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
    product_template_df = product_template_df_fun(product_template_json)


    complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)

    return complete_costo_df


## Pruebas

In [21]:
api_params = api_params_func()
purchase_lines_json, product_template_json = api_call_func(api_params)

purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
product_template_df = product_template_df_fun(product_template_json)

complete_costo_df = complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)


In [22]:
from algoritmo_ventas_mes import ventas_mes_func
ventas_enero = ventas_mes_func(1)
ventas_febrero = ventas_mes_func(2)
ventas_marzo = ventas_mes_func(3)
ventas_abril = ventas_mes_func(4)

wep = costo_func()

In [23]:
ventas_enero

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,company_id,partner_id,invoice_user_id,fact_line_id,product_id,quantity,price_unit,discount,price_subtotal,salesperson_id
0,114,F1-CC/2024/00001,2024-01-02,posted,Shop/0005,PdV,10,out_invoice,<NA>,<NA>,...,1,15271,226,302,8068,20.0,6.47,0.0,129.40,218
1,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,311,9445,3.0,36.03,0.0,108.09,218
2,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,312,9539,3.0,37.98,0.0,113.94,218
3,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,313,12520,1.0,209.99,0.2,167.99,218
4,122,F1-CC/2024/00003,2024-01-02,posted,Shop/0006,PdV,13,out_invoice,<NA>,<NA>,...,1,15468,226,332,8101,20.0,7.56,0.0,151.20,219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,1,13732,229,62470,11834,2.0,143.44,0.2,229.50,221
10242,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,1,13732,229,62471,8290,2.0,10.79,0.2,17.26,221
10243,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,1,13732,229,62472,8274,2.0,10.51,0.2,16.82,221
10244,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,1,13732,229,62473,11127,1.0,102.29,0.2,81.83,221


In [24]:
costo = wep[['product_id', 'cost_type', 'cost_unit']]
costo.head()

,product_id,cost_type,cost_unit
0,29719,REPS,-107.73
1,29540,REPS,-421.55
2,29764,REPS,-1120.60
3,29871,REPS,-39439.66
4,27326,REPS,-151.57


In [25]:
ventas_enero.head()

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,company_id,partner_id,invoice_user_id,fact_line_id,product_id,quantity,price_unit,discount,price_subtotal,salesperson_id
0,114,F1-CC/2024/00001,2024-01-02,posted,Shop/0005,PdV,10,out_invoice,<NA>,<NA>,...,1,15271,226,302,8068,20.0,6.47,0.0,129.40,218
1,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,311,9445,3.0,36.03,0.0,108.09,218
2,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,312,9539,3.0,37.98,0.0,113.94,218
3,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,1,15413,226,313,12520,1.0,209.99,0.2,167.99,218
4,122,F1-CC/2024/00003,2024-01-02,posted,Shop/0006,PdV,13,out_invoice,<NA>,<NA>,...,1,15468,226,332,8101,20.0,7.56,0.0,151.20,219


In [26]:
enero = ventas_enero.merge(costo, how='left', on='product_id')
enero['cost_subtotal'] = enero['quantity'] * enero['cost_unit']
enero['utilidad'] = enero['price_subtotal'] + enero['cost_subtotal']

enero

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,product_id,quantity,price_unit,discount,price_subtotal,salesperson_id,cost_type,cost_unit,cost_subtotal,utilidad
0,114,F1-CC/2024/00001,2024-01-02,posted,Shop/0005,PdV,10,out_invoice,<NA>,<NA>,...,8068,20.0,6.47,0.0,129.40,218,REPS,-4.17,-83.40,46.00
1,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,9445,3.0,36.03,0.0,108.09,218,Compra,-22.17,-66.51,41.58
2,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,9539,3.0,37.98,0.0,113.94,218,Compra,-23.37,-70.11,43.83
3,117,F1-CC/2024/00002,2024-01-02,posted,Shop/0004,PdV,9,out_invoice,<NA>,<NA>,...,12520,1.0,209.99,0.2,167.99,218,Compra,-146.04,-146.04,21.95
4,122,F1-CC/2024/00003,2024-01-02,posted,Shop/0006,PdV,13,out_invoice,<NA>,<NA>,...,8101,20.0,7.56,0.0,151.20,219,Compra,-4.65,-93.00,58.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,11834,2.0,143.44,0.2,229.50,221,Compra,-100.42,-200.84,28.66
10242,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,8290,2.0,10.79,0.2,17.26,221,Compra,-6.64,-13.28,3.98
10243,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,8274,2.0,10.51,0.2,16.82,221,Compra,-6.47,-12.94,3.88
10244,16489,F2-CC/2024/02153,2024-01-31,posted,PdV SJC/2220,PdV,3899,out_invoice,<NA>,<NA>,...,11127,1.0,102.29,0.2,81.83,221,REPS,-64.95,-64.95,16.88


In [27]:
enero.loc[enero['price_unit'] < enero['cost_unit'].abs()]

,fact_doc_id,name,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,...,product_id,quantity,price_unit,discount,price_subtotal,salesperson_id,cost_type,cost_unit,cost_subtotal,utilidad
286,647,F1-CC/2024/00049,2024-01-03,posted,Shop/0080,PdV,202,out_invoice,<NA>,<NA>,...,9051,6.0,26.78,0.0,160.68,218,Compra,-32.96,-197.76,-37.08
655,1240,F2-CC/2024/00167,2024-01-04,posted,PdV SJC/0213,PdV,396,out_invoice,<NA>,<NA>,...,9051,4.0,26.78,0.0,107.12,227,Compra,-32.96,-131.84,-24.72
1018,1894,F1-CC/2024/00206,2024-01-05,posted,Shop/0112,PdV,276,out_invoice,<NA>,<NA>,...,10031,5.0,32.50,0.0,162.50,218,REPS,-33.00,-165.00,-2.50
1150,2120,F2-CC/2024/00252,2024-01-05,posted,PdV SJC/0242,PdV,448,out_invoice,<NA>,<NA>,...,9051,2.0,26.78,0.0,53.56,233,Compra,-32.96,-65.92,-12.36
1619,2995,F2-VS/2024/00025,2024-01-08,posted,S01025,Ventas,<NA>,out_invoice,<NA>,<NA>,...,9980,15.0,26.00,0.0,390.00,213,REPS,-31.60,-474.00,-84.00
1683,3086,F2-VS/2024/00029,2024-01-08,posted,S01069,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14862,4.0,627.05,0.0,2508.20,213,REPS,-742.35,-2969.40,-461.20
1853,3236,F1-VS/2024/00062,2024-01-08,posted,S01181,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14864,20.0,60.00,0.0,1200.00,210,REPS,-418.03,-8360.60,-7160.60
1898,3270,F1-CC/2024/00327,2024-01-08,posted,PdV CSL/0322,PdV,780,out_invoice,<NA>,<NA>,...,9051,11.0,26.78,0.0,294.58,212,Compra,-32.96,-362.56,-67.98
1972,3368,F2-CC/2024/00436,2024-01-08,posted,PdV SJC/0437,PdV,791,out_invoice,<NA>,<NA>,...,9051,8.0,26.78,0.0,214.24,216,Compra,-32.96,-263.68,-49.44
2016,3396,F1-VS/2024/00085,2024-01-08,posted,S01229,Ventas,<NA>,out_invoice,<NA>,<NA>,...,14839,3.0,234.00,0.0,702.00,210,REPS,-6036.79,-18110.37,-17408.37
